本章将介绍几种强大的工具，可以让你朝着针对困难问题来开发最先进模型这一目标更近一步。利用 Keras函数式  API，你可以构建类图（graph-like）模型、在不同的输入之间共享某一层，并且还可以像使用 Python函数一样使用   Keras模型。Keras回调函数和   TensorBoard基于浏览器的可视化工具，让你可以在训练过程中监控模型。我们还会讨论其他几种最佳实践，包括批标准化、残差连接、超参数优化和模型集成

### 7.1　不用 Sequential模型的解决方案：Keras函数式   API

Sequential模型类就能够涵盖许多主题和实际应用。但有些情况下这种假设过于死板。有些网络需要多个独立的输入，有些网络则需要多个输出，而有些网络在层与层之间具有内部分支，这使得网络看起来像是层构成的图（），而不是层的线性堆叠。   


三个重要的使用案例（多输入模型、多输出模型和类图模型），只用  Keras中的Sequential模型类是无法实现的。但是还有另一种更加通用、更加灵活的使用    Keras的方式，就是函数式API（functional API）。本节将会详细介绍函数式 API是什么、能做什么以及如何使用它。


### 7.1.1　函数式 API简介
使用函数式 API，你可以直接操作张量，也可以把层当作函数来使用，接收张量并返回张量（因此得名函数式 API）。

In [1]:
from tensorflow import keras
from tensorflow.keras import Input, layers
from tensorflow.keras.models import Sequential,Model

input_tensor= Input(shape=(32,))
dense= layers.Dense(32, activation= 'relu')
output_tensor= dense(input_tensor)

In [3]:
input_tensor= Input(shape=(64,))
x= layers.Dense(32, activation= 'relu')(input_tensor)
x=layers.Dense(32,activation='relu')(x)
output_tensor=layers.Dense(10,activation='softmax')(x)

model= Model(input_tensor, output_tensor)
model.summary()



Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_6 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


### 7.1.2　多输入模型

函数式 API可用于构建具有多个输入的模型。通常情况下，这种模型会在某一时刻用一个可以组合多个张量的层将不同的输入分支合并，张量组合方式可能是相加、连接等。这通常利用 Keras的合并运算来实现，比如keras.layers.add、keras.layers.concatenate等。我们来看一个非常简单的多输入模型示例——一个问答模型。    

典型的问答模型有两个输入：一个自然语言描述的问题和一个文本片段（比如新闻文章），后者提供用于回答问题的信息。然后模型要生成一个回答，在最简单的情况下，这个回答只包含一个词，可以通过对某个预定义的词表做 softmax得到（见图 7-6）。   

我们设置了两个独立分支，首先将文本输入和问题输入分别编码为表示向量，然后连接这些向量，最后，在连接好的表示上添加一个 softmax分类器。

In [5]:
text_vocabulary_size=10000
question_vocabulary_size=10000
answer_vocabulary_size=500

text_input=Input(shape=(None,),dtype='int32',name='text')

embedded_text= layers.Embedding(text_vocabulary_size,64)(text_input)# 将输入嵌入长度为64的向量

encoded_text=layers.LSTM(32)(embedded_text) # 利用LSTM将向量编码为单个向量

question_input= Input(shape=(None,),
                     dtype='int32',
                     name='question') # 对问题进行相同的处理
embedded_question= layers.Embedding(question_vocabulary_size,32)(question_input)# 将输入嵌入长度为64的向量

encoded_question=layers.LSTM(16)(embedded_question) # 利用LSTM将向量编码为单个向量

concatenated= layers.concatenate([encoded_text,encoded_question],
                                axis=-1) # 将编码后的问题和文本连接起来

answer=layers.Dense(answer_vocabulary_size,
                   activation='softmax')(concatenated)

model= Model([text_input,question_input],answer)

model.compile(optimizer= 'rmsprop',
             loss='categorical_crossentropy',
             metrics=['acc'])


接下来要如何训练这个双输入模型呢？有两个可用的     API：我们可以向模型输入一个由Numpy数组组成的列表，或者也可以输入一个将输入名称映射为     Numpy数组的字典。当然，只有输入具有名称时才能使用后一种方法。

In [6]:
import numpy as np

num_samples=1000
max_length=100

text= np.random.randint(1,text_vocabulary_size,
                       size=(num_samples,max_length))

question= np.random.randint(1,question_vocabulary_size,
                       size=(num_samples,max_length))

answers= np.random.randint(answer_vocabulary_size,
                          size=(num_samples))

answers= keras.utils.to_categorical(answers,answer_vocabulary_size)

model.fit([text,question],answers,epochs=10,batch_size=128)

model.fit({'text':text,'question':question},answers,epochs=10,batch_size=128)


Epoch 1/10
8/8 [==============================] - 3s 51ms/step - loss: 6.2148 - acc: 0.0000e+00
Epoch 2/10
8/8 [==============================] - 0s 49ms/step - loss: 6.1992 - acc: 0.0474
Epoch 3/10
8/8 [==============================] - 0s 49ms/step - loss: 6.1601 - acc: 0.0088
Epoch 4/10
8/8 [==============================] - 0s 51ms/step - loss: 6.0609 - acc: 0.0031
Epoch 5/10
8/8 [==============================] - 0s 48ms/step - loss: 6.0012 - acc: 0.0101
Epoch 6/10
8/8 [==============================] - 0s 51ms/step - loss: 5.9462 - acc: 0.0093
Epoch 7/10
8/8 [==============================] - 0s 55ms/step - loss: 5.8605 - acc: 0.0128
Epoch 8/10
8/8 [==============================] - 0s 53ms/step - loss: 5.7593 - acc: 0.0293
Epoch 9/10
8/8 [==============================] - 0s 52ms/step - loss: 5.6770 - acc: 0.0352
Epoch 10/10
8/8 [==============================] - 0s 52ms/step - loss: 5.5946 - acc: 0.0372
Epoch 1/10
8/8 [==============================] - 2s 54ms/step - loss: 5.50

### 7.1.3　多输出模型

我们还可以使用函数式  API来构建具有多个输出（或多头）的模型。一个简单的例子就是一个网络试图同时预测数据的不同性质，比如一个网络，输入某个匿名人士的一系列社交媒体发帖，然后尝试预测那个人的属性，比如年龄、性别和收入水平

In [10]:
vocabulary_size = 50000
num_income_groups = 10
posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(256, vocabulary_size)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

In [12]:
age_prediction= layers.Dense(1,name='age')(x)

income_prediction= layers.Dense(num_income_groups,
                               activation='softmax',
                               name='income')(x)
gender_prediction=layers.Dense(1, activation='sigmoid',name='gender')(x)

model=Model(posts_input,[age_prediction,income_prediction,gender_prediction])

重要的是，训练这种模型需要能够对网络的各个头指定不同的损失函数，例如，年龄预测是标量回归任务，而性别预测是二分类任务，二者需要不同的训练过程。但是，梯度下降要求将一个标量最小化，所以为了能够训练模型，我们必须将这些损失合并为单个标量。合并不同损失最简单的方法就是对所有损失求和。在  Keras中，你可以在编译时使用损失组成的列表或字典来为不同输出指定不同损失，然后将得到的损失值相加得到一个全局损失，并在训练过程中将这个损失最小化。



In [13]:
#多输出模型的编译选项：多重损失
model.compile(optimizer='rmsprop',
             loss=['mse','categorical_crossentropy','binary_crossentropy'])

model.compile(optimizer='rmsprop',loss={'age':'mse',
                                       'income':'categorical_crossentropy',
                                       'gender':'binary_crossentropy'})

严重不平衡的损失贡献会导致模型表示针对单个损失值最大的任务优先进行优化，而不考虑其他任务的优化。为了解决这个问题，我们可以为每个损失值对最终损失的贡献分配不同大小的重要性。如果不同的损失值具有不同的取值范围，那么这一方法尤其有用。比如，用于年龄回归任务的均方误差（ MSE）损失值通常在 3~5左右，而用于性别分类任务的交叉熵损失值可能低至 0.1。在这种情况下，为了平衡不同损失的贡献，我们可以让交叉熵损失的权重取 10，而 MSE损失的权重取  0.5。

In [14]:
#多输出模型的编译选项：损失加权
model.compile(optimizer='rmsprop',
             loss=['mse','categorical_crossentropy','binary_crossentropy'],
             loss_weights=[0.25,1.,10.])

model.compile(optimizer='rmsprop',loss={'age':'mse',
                                       'income':'categorical_crossentropy',
                                       'gender':'binary_crossentropy'},
             loss_weights={'age':0.25,
                          'income':1.,
                          'gender':10.})

In [ ]:
# 与多输入模型相同，多输出模型的训练输入数据可以是 Numpy数组组成的列表或字典。

model.fit(posts, [age_targets, income_targets,gender_targets],
         epochs=10,batch_size=64)
model.fit(posts, {'age':age_targets,
                  'income':income_targets,
                  'gender':gender_targets},
          epochs=10,batch_size=64)

### 7.1.4　层组成的有向无环图
利用函数式  API，我们不仅可以构建多输入和多输出的模型，而且还可以实现具有复杂的内部拓扑结构的网络。 Keras中的神经网络可以是层组成的任意有向无环图（directed    acyclicgraph）。无环（acyclic）这个限定词很重要，即这些图不能有循环。张量  x不能成为生成 x的某一层的输入。唯一允许的处理循环（即循环连接）是循环层的内部循环。

一些常见的神经网络组件都以图的形式实现。两个著名的组件是 Inception模块和残差连接。为了更好地理解如何使用函数式 API来构建层组成的图，我们来看一下如何用 Keras实现这二者。



#### 1. Inception模块
 Inception模块最基本的形式包含 3~4个分支，首先是一个   1×1的卷积，然后是一个  3×3的卷积，最后将所得到的特征连接在一起。这种设置有助于网络分别学习空间特征和逐通道的特征，这比联合学习这两种特征更加有效

In [ ]:
# 这个例子假设我们有一个四维输入张量x。

branch_a=layers.Conv2D(128,1,activation='relu',strides=2)(x)
branch_b=layers.Conv2D(128,1,activation='relu')(x)
branch_b=layers.Conv2D(128,3,activation='relu',strides=2)(branch_b)

branch_c=layers.AveragePooling2D(3, strides=2)(x)
branch_c=layers.Conv2D(128,3,activation='relu',strides=2)(branch_c)

branch_d=layers.Conv2D(128,1,activation='relu')(x)
branch_d=layers.Conv2D(128,1,activation='relu')(branch_d)
branch_d=layers.Conv2D(128,3,activation='relu',strides=2)(branch_d)

output= layers.concatenate([branch_a,
                           branch_b,branch_c,branch_d],axis=-1)



#### 2.残差连接

残差连接解决了困扰所有大规模深度学习模型的两个共性问题：梯度消失和表示瓶颈。通常来说，向任何多于 10层的模型中添加残差连接，都可能会有所帮助。   

残差连接是让前面某层的输出作为后面某层的输入，从而在序列网络中有效地创造了一条捷径。前面层的输出没有与后面层的激活连接在一起，而是与后面层的激活相加（这里假设两个激活的形状相同）。如果它们的形状不同，我们可以用一个线性变换将前面层的激活改变成目标形状（例如，这个线性变换可以是不带激活的 Dense层；对于卷积特征图，可以是不带激活1×1卷积）       

如果特征图的尺寸相同，在 Keras中实现残差连接的方法如下，用的是恒等残差连接（identity residual connection）。这个例子假设我们有一个四维输入张量x

In [ ]:
y=layers.Conv2D(128,3,activation='relu',padding='same')(x)
y=layers.Conv2D(128,3,activation='relu',padding='same')(y)
y=layers.Conv2D(128,3,activation='relu',padding='same')(y)

y=layers.add([y,x])


如果特征图的尺寸不同，实现残差连接的方法如下，用的是线性残差连接（   linear  residual connection）。同样，假设我们有一个四维输入张量x。

In [ ]:
y=layers.Conv2D(128,3,activation='relu',padding='same')(x)
y=layers.Conv2D(128,3,activation='relu',padding='same')(y)
y=layers.MaxPooling2D(2, strides=2)(y)

residual= layers.Conv2D(128,1, strides=2, padding='same')(x)# 使用 1x1卷积，将原始x张量线性下采样与具有相同的形状

y=layers.add([y,residual])

#### 深度学习中的表示瓶颈
在Sequential模型中，每个连续的表示层都构建于前一层之上，这意味着它只能访问前一层激活中包含的信息。如果某一层太小（比如特征维度太低），那么模型将会受限于该层激活中能够塞入多少信息。

你可以通过类比信号处理来理解这个概念：假设你有一条包含一系列操作的音频处理流水线，每个操作的输入都是前一个操作的输出，如果某个操作将信号裁剪到低频范围（比如0~15 kHz），那么下游操作将永远无法恢复那些被丢弃的频段。任何信息的丢失都是永久性的。残差连接可以将较早的信息重新注入到下游数据中，从而部分解决了深度学习模型的这一问题。

#### 深度学习中的梯度消失
反向传播是用于训练深度神经网络的主要算法，其工作原理是将来自输出损失的反馈信号向下传播到更底部的层。如果这个反馈信号的传播需要经过很多层，那么信号可能会变得非常微弱，甚至完全丢失，导致网络无法训练。这个问题被称为梯度消失（vanishing gradient）。

深度网络中存在这个问题，在很长序列上的循环网络也存在这个问题。在这两种情况下，反馈信号的传播都必须通过一长串操作。我们已经知道LSTM层是如何在循环网络中解决这个问题的：它引入了一个携带轨道（carry track），可以在与主处理轨道平行的轨道上传播信息。残差连接在前馈深度网络中的工作原理与此类似，但它更加简单：它引入了一个纯线性的信息携带轨道，与主要的层堆叠方向平行，从而有助于跨越任意深度的层来传播梯度。

### 7.1.5　共享层权重

函数式 API还有一个重要特性，那就是能够多次重复使用一个层实例。如果你对一个层实例调用两次，而不是每次调用都实例化一个新层，那么每次调用可以重复使用相同的权重。这样你可以构建具有共享分支的模型，即几个分支全都共享相同的知识并执行相同的运算。也就是说，这些分支共享相同的表示，并同时对不同的输入集合学习这些表示。

举个例子，假设一个模型想要评估两个句子之间的语义相似度。这个模型有两个输入（需要比较的两个句子），并输出一个范围在 0~1的分数，0表示两个句子毫不相关，   1表示两个句子完全相同或只是换一种表述。这种模型在许多应用中都很有用，其中包括在对话系统中删除重复的自然语言查询。

在这种设置下，两个输入句子是可以互换的，因为语义相似度是一种对称关系，      A相对于 B的相似度等于   B相对于  A的相似度。因此，学习两个单独的模型来分别处理两个输入句子是没有道理的。相反，你需要用一个 LSTM层来处理两个句子。这个  LSTM层的表示（即它的权重）是同时基于两个输入来学习的。我们将其称为连体    LSTM（Siamese  LSTM）或共享LSTM（shared  LSTM）模型。

In [ ]:
lstm= layers.LSTM(32)

left_input= Input(shape=(None,128))
left_output= lstm(left_input)

right_input= Input(shape=(None,128))
left_output= lstm(right_input)

merged= layers.concatenate([left_input,right_input],axis=-1)
predictions= layers.Dense(1, activation='sigmoid')(merged)

model= Model([left_input,right_input],predictions)
model.fit([left_data,right_data],targets)




### 7.1.6　将模型作为层
重要的是，在函数式 API中，可以像使用层一样使用模型。实际上，你可以将模型看作“更大的层”。Sequential类和Model类都是如此。这意味着你可以在一个输入张量上调用模型，并得到一个输出张量

In [ ]:
y = model(x)
# 如果模型具有多个输入张量和多个输出张量，那么应该用张量列表来调用模型。
y1,y2= model([x1,x2])


通过重复使用模型实例可以构建一个简单的例子，就是一个使用双摄像头作为输入的视觉模型：两个平行的摄像头，相距几厘米（一英寸）。这样的模型可以感知深度，这在很多应用中都很有用。你不需要两个单独的模型从左右两个摄像头中分别提取视觉特征，然后再将二者合并。这样的底层处理可以在两个输入之间共享，即通过共享层（使用相同的权重，从而共享相同的表示）来实现。在 Keras中实现连体视觉模型（共享卷积基）的代码如下所示。

In [ ]:
from tensorflow.keras import applications
xception_base= applications.Xception(weights=None,
                                    include_top=False)

left_input= Input(shape= (250, 250,3))
right_input= Input(shape=(250,250,3))

left_features= xception_base(left_input)
right_input=xception_base(right_input) # 对相同的视觉模型调用两次

merged_features= layers.concatenate([left_features,
                                    right_input],axis =-1)



### 7.1.7　小结

以上就是对 Keras函数式   API的介绍，它是构建高级深度神经网络架构的必备工具。本节我们学习了以下内容。

如果你需要实现的架构不仅仅是层的线性堆叠，那么不要局限于Sequential   API。    
如何使用  Keras函数式   API来构建多输入模型、多输出模型和具有复杂的内部网络拓扑结构的模型。   
如何通过多次调用相同的层实例或模型实例，在不同的处理分支之间重复使用层或模型的权重  